# SCORE DISTRIBUTION


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO

cleaned_data_api = 'https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download'
tinh_api = 'https://andyanh.id.vn/index.php/s/zbHTAjksBekNB4M/download'

def fetch_csv_from_api(api_url):
    from datetime import datetime, timedelta
    import os
    
    cache_file = 'cleaned_data_cache.csv' if 'AQrka' in api_url else 'tinh_cache.csv'
    cache_timeout = timedelta(hours=24)
    
    if os.path.exists(cache_file):
        modified_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - modified_time < cache_timeout:
            print(f"Đang tải dữ liệu từ cache {cache_file}...")
            return pd.read_csv(cache_file)
    
    print(f"Đang tải dữ liệu từ API {api_url}...")
    response = requests.get(api_url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv(cache_file, index=False)
        return df
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Tải dữ liệu từ API
try:
    cleaned_data_df = fetch_csv_from_api(cleaned_data_api)
    tinh_df = fetch_csv_from_api(tinh_api)
    print("Đã tải dữ liệu thành công từ API")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu từ API: {e}")
    print("Không thể tải dữ liệu. Vui lòng kiểm tra kết nối internet và thử lại.")
    exit()

# Tạo bins cho điểm số từ 1-10
bins = list(range(1, 12))
labels = [f'{i}' for i in range(1, 11)]

# Tính phân phối điểm cho từng năm và môn học
def calculate_score_distribution(df, year, subject):
    year_data = df[df['Year'] == year][subject]
    # Lọc bỏ điểm -1 (điểm không hợp lệ)
    year_data = year_data[year_data >= 0]
    hist, _ = np.histogram(year_data, bins=bins)
    return (hist / len(year_data)) * 100

# Danh sách các môn học
subjects = ['Toan', 'Van', 'Ly', 'Sinh', 'Ngoai ngu', 'Hoa', 'Lich su', 'Dia ly', 'GDCD']

# Tạo DataFrame để lưu kết quả cho từng môn
results_dict = {'Điểm': labels}

for subject in subjects:
    dist_2018 = calculate_score_distribution(cleaned_data_df, 2018, subject)
    dist_2019 = calculate_score_distribution(cleaned_data_df, 2019, subject)
    
    results_dict[f'Tỉ lệ {subject} 2018 (%)'] = np.round(dist_2018, 2)
    results_dict[f'Tỉ lệ {subject} 2019 (%)'] = np.round(dist_2019, 2)

results = pd.DataFrame(results_dict)

# Xuất ra file CSV
output_file = 'phan_phoi_diem.csv'
results.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\nĐã xuất dữ liệu ra file: {output_file}")

Đang tải dữ liệu từ API https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download...
Đang tải dữ liệu từ cache tinh_cache.csv...
Đã tải dữ liệu thành công từ API

Đã xuất dữ liệu ra file: phan_phoi_diem.csv
